In [2]:
from langchain.document_loaders import HuggingFaceDatasetLoader
dataset_name = "wikipedia"
page_content_column = "text"
name = "20220301.simple"


loader = HuggingFaceDatasetLoader(dataset_name, page_content_column, name)
docs = loader.load()

Found cached dataset wikipedia (/Users/deanchanter/.cache/huggingface/datasets/wikipedia/20220301.simple/2.0.0/aa542ed919df55cc5d3347f42dd4521d05ca68751f50dbc32bae2a7f1e167559)


  0%|          | 0/1 [00:00<?, ?it/s]

In [3]:
from  langchain.schema import Document
import json
from typing import Iterable

def save_docs_to_jsonl(array:Iterable[Document], file_path:str)->None:
    with open(file_path, 'w') as jsonl_file:
        for doc in array:
            jsonl_file.write(doc.json() + '\n')

In [4]:
save_docs_to_jsonl(docs,'wiki.jsonl')

In [10]:
# import
#from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings
#from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
#from langchain.document_loaders import TextLoader
#from langchain.embeddings.openai import OpenAIEmbeddings
import os
#from app import get_secrets
#os.environ['OPENAI_API_KEY'] = get_secrets(llm='OPENAI')
#embedding_function =OpenAIEmbeddings()
# Load model directly
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("BAAI/bge-large-en")

from langchain.text_splitter import TokenTextSplitter
from langchain.embeddings import HuggingFaceBgeEmbeddings

model_name = "BAAI/bge-small-en"
#model_kwargs = {'device': 'cuda'}
#encode_kwargs = {'normalize_embeddings': True} # set True to compute cosine similarity
bge_hf = HuggingFaceBgeEmbeddings(
    model_name=model_name,
 #   model_kwargs=model_kwargs,
 #   encode_kwargs=encode_kwargs
)

# split it into chunks
text_splitter = TokenTextSplitter.from_huggingface_tokenizer(tokenizer,chunk_size=200, chunk_overlap=10)
#text_splitter = TokenTextSplitter(chunk_size=1000, chunk_overlap=25)
docs_split = text_splitter.transform_documents(docs)
#docs_split=docs
#print(docs_split[0])
embedding_function= bge_hf


# load it into Chroma
db = Chroma.from_documents(docs_split, embedding_function,persist_directory="./chroma_db_wiki")
db.persist()


--- Logging error ---
Traceback (most recent call last):
  File "/Users/deanchanter/Documents/GitHub/comma-chameleons/env/lib/python3.11/site-packages/chromadb/db/mixins/embeddings_queue.py", line 263, in _notify_one
    sub.callback([embedding])
  File "/Users/deanchanter/Documents/GitHub/comma-chameleons/env/lib/python3.11/site-packages/chromadb/segment/impl/metadata/sqlite.py", line 393, in _write_metadata
    self._insert_record(cur, record, True)
  File "/Users/deanchanter/Documents/GitHub/comma-chameleons/env/lib/python3.11/site-packages/chromadb/segment/impl/metadata/sqlite.py", line 227, in _insert_record
    self._update_metadata(cur, id, record["metadata"])
  File "/Users/deanchanter/Documents/GitHub/comma-chameleons/env/lib/python3.11/site-packages/chromadb/segment/impl/metadata/sqlite.py", line 253, in _update_metadata
    cur.execute(sql, params)
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  Fil

In [2]:
#Load db
from langchain.vectorstores.chroma import Chroma
from langchain.embeddings import HuggingFaceBgeEmbeddings

model_name = "BAAI/bge-small-en"
#model_kwargs = {'device': 'cuda'}
#encode_kwargs = {'normalize_embeddings': True} # set True to compute cosine similarity
bge_hf = HuggingFaceBgeEmbeddings(
    model_name=model_name,
)
db = Chroma(embedding_function = bge_hf,persist_directory="./chroma_db")
db.persist()


In [6]:
db.embeddings

HuggingFaceBgeEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': True}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': True, 'pooling_mode_mean_tokens': False, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
), model_name='BAAI/bge-small-en', cache_folder=None, model_kwargs={}, encode_kwargs={}, query_instruction='Represent this question for searching relevant passages: ')

In [1]:
# query it
query = "Who is Allen Turing"
docs_q = db.similarity_search('what is memory',search_type="mmr", search_kwargs={'k': 5, 'fetch_k': 50})

# print results
for d in docs_q:
    print(d.page_content)

NameError: name 'db' is not defined

In [ ]:
'''
from ibm_watson_machine_learning.foundation_models.extensions.langchain import WatsonxLLM
from ibm_watson_machine_learning.foundation_models.extensions.langchain import WatsonxLLM
from ibm_watson_machine_learning.foundation_models.utils.enums import ModelTypes, DecodingMethods
from ibm_watson_machine_learning.metanames import GenTextParamsMetaNames as GenParams
from ibm_watson_machine_learning.foundation_models import Model
from langchain.memory import ConversationBufferMemory, ConversationSummaryMemory
from langchain.prompts import PromptTemplate
#from langchain.prompts.pipeline import PipelinePromptTemplate
from langchain.chains import LLMChain, ConversationChain, ConversationalRetrievalChain
from langchain.chains import RetrievalQA

params = {
    GenParams.MAX_NEW_TOKENS: 1000,
    GenParams.MIN_NEW_TOKENS: 1,
    GenParams.DECODING_METHOD: DecodingMethods.GREEDY,
    GenParams.TEMPERATURE: 0.8,
    GenParams.TOP_K: 50,
    GenParams.TOP_P: .95
}
credentials = {
    'url': "https://us-south.ml.cloud.ibm.com",
    'apikey' : '5Oz7Ma1ZCctfI0I9FU1kbZC6YRWm9jRKddtL-8mJYREx'
}

project_id = '0ec8f431-3b27-41a3-a540-98d7f62ca1ad'


llama = ModelTypes.LLAMA_2_70B_CHAT

llama_model = Model(
model_id=llama,
params=params,
credentials=credentials,
project_id=project_id)
llm = WatsonxLLM(model=llama_model)


prompt_template = """Answer the question based only on the following context:

{context}

Question: {question}"""

PROMPT = PromptTemplate.from(
    template=prompt_template, input_variables=["context", "question"]
)
chain_type_kwargs = {"prompt": PROMPT}
#memory = ConversationBufferMemory(memory_key='chat_history')
#retriever = db.as_retriever()
# Retrieve more documents with higher diversity- useful if your dataset has many similar documents
#retriever=db.as_retriever(search_type="mmr", search_kwargs={'k': 8, 'lambda_mult': 0.25})

# Fetch more documents for the MMR algorithm to consider, but only return the top 5
retriever=db.as_retriever(search_type="mmr", search_kwargs={'k': 5, 'fetch_k': 50})

# Only retrieve documents that have a relevance score above a certain threshold
#retriever= db.as_retriever(search_type="similarity_score_threshold", search_kwargs={'score_threshold': 0.8})

# Only get the single most similar document from the dataset
#retriever=db.as_retriever(search_kwargs={'k': 5})

# Use a filter to only retrieve documents from a specific paper 
#docsearch.as_retriever(search_kwargs={'filter': {'paper_title':'GPT-4 Technical Report'}})
memory = ConversationSummaryMemory(llm=llm,memory_key="chat_history",return_messages=True)

conv = ConversationalRetrievalChain.from_llm(llm, retriever=retriever, memory=memory, prompt = PROMPT)

#qa_s = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retriever,chain_type_kwargs=chain_type_kwargs)
#chain = LLMChain(llm=llm, prompt=prompt, verbose=True, output_key='output', memory=memory)
'''

In [ ]:
# Helper function for printing docs

def pretty_print_docs(docs):
    print(f"\n{'-' * 100}\n".join([f"Document {i+1}:\n\n" + d.page_content for i, d in enumerate(docs)]))

In [38]:

from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor

#LLM Compressor....too slow
"""
retriever = db.as_retriever(search_type="mmr", search_kwargs={'k': 8, 'lambda_mult': 0.25})
llm = llm_llama_temp_0p1
compressor = LLMChainExtractor.from_llm(llm)
compression_retriever = ContextualCompressionRetriever(base_compressor=compressor, base_retriever=retriever)

compressed_docs = compression_retriever.get_relevant_documents('What is a chain in langchain?')
pretty_print_docs(compressed_docs)
"""

Document 1:

* "Chains"
* "Chain"
* "chaining LLMs"
* "LangChain provides the Chain interface"
* "a Chain very generically as a sequence of calls to components, which can include other chains"
* "BaseChain"
* "ChainValues"
* "abstract class BaseChain"
* "memory"
* "call"
* "runManager"
* "input keys"
* "output keys"

Note: The extracted parts are not edited, they are copied as is from the context.
----------------------------------------------------------------------------------------------------
Document 2:

* Debugging chains
* Adding memory (state)
* Chain object
* LangChain Expression Language (LCEL)
* Modules
* Memory object
* Stateful

Please note that the extracted parts are not necessarily the only relevant parts, and the question may require additional information from the context to be fully answered.
----------------------------------------------------------------------------------------------------
Document 3:

* Debugging chains
* Chainobject
* Settingverbosetotrue
* Conve

In [8]:
from typing import List
from langchain import LLMChain
from pydantic import BaseModel, Field
from langchain.prompts import PromptTemplate
from langchain.output_parsers import PydanticOutputParser
from langchain.agents.agent_toolkits import create_conversational_retrieval_agent
from langchain.agents.agent_toolkits import create_retriever_tool
from ibm_watson_machine_learning.foundation_models.extensions.langchain import WatsonxLLM
from ibm_watson_machine_learning.foundation_models.utils.enums import ModelTypes, DecodingMethods
from ibm_watson_machine_learning.metanames import GenTextParamsMetaNames as GenParams
from ibm_watson_machine_learning.foundation_models import Model
from langchain.memory import ConversationBufferMemory, ConversationSummaryMemory
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain.retrievers import ContextualCompressionRetriever
#from langchain.retrievers.document_compressors import LLMChainExtractor
from langchain.retrievers.document_compressors import EmbeddingsFilter

# Helper function for printing docs

def pretty_print_docs(docs):
    print(f"\n{'-' * 100}\n".join([f"Document {i+1}:\n\n" + d.page_content for i, d in enumerate(docs)]))



params_temp_0p1 = {
    GenParams.MAX_NEW_TOKENS: 1000,
    GenParams.MIN_NEW_TOKENS: 1,
    GenParams.DECODING_METHOD: DecodingMethods.GREEDY,
    GenParams.TEMPERATURE: 0.1,
    GenParams.TOP_K: 50,
    GenParams.TOP_P: .95
}
credentials = {
    'url': "https://us-south.ml.cloud.ibm.com",
    'apikey' : '5Oz7Ma1ZCctfI0I9FU1kbZC6YRWm9jRKddtL-8mJYREx'
}

project_id = '0ec8f431-3b27-41a3-a540-98d7f62ca1ad'


llama = ModelTypes.LLAMA_2_70B_CHAT

llama_model_temp_0p1 = Model(
model_id=llama,
params=params_temp_0p1,
credentials=credentials,
project_id=project_id)
llm_llama_temp_0p1 = WatsonxLLM(model=llama_model_temp_0p1)

# Output parser will split the LLM result into a list of queries
class LineList(BaseModel):
    # "lines" is the key (attribute name) of the parsed output
    lines: List[str] = Field(description="Lines of text")


class LineListOutputParser(PydanticOutputParser):
    def __init__(self) -> None:
        super().__init__(pydantic_object=LineList)

    def parse(self, text: str) -> LineList:
        lines = text.strip().split("\n")
        return LineList(lines=lines)


output_parser = LineListOutputParser()

QUERY_PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""You are an AI language model assistant. Your task is to generate three
    different versions of the given user question to retrieve relevant documents from a vector 
    database. By generating multiple perspectives on the user question, your goal is to help
    the user overcome some of the limitations of the distance-based similarity search. 
    Provide these alternative questions seperated by newlines. The default context of the topic is langchain.
    Original question: {question}""",
)

base_retriever = db.as_retriever()



#embeddings = bge_hf
embeddings = db.embeddings
embeddings_filter = EmbeddingsFilter(embeddings=embeddings, similarity_threshold=0.85)
compression_retriever = ContextualCompressionRetriever(base_compressor=embeddings_filter, base_retriever=base_retriever)

#too slow
#compressor = LLMChainExtractor.from_llm(llm_llama_temp_0p1)
#compression_retriever = ContextualCompressionRetriever(base_compressor=compressor, base_retriever=base_retriever)


retriever_llm_chain = LLMChain(llm=llm_llama_temp_0p1, prompt=QUERY_PROMPT, output_parser=output_parser)
multi_retriever = MultiQueryRetriever(
    retriever=compression_retriever, llm_chain=retriever_llm_chain, parser_key="lines"
)  # "lines" is the key (attribute name) of the parsed output

In [9]:
# Run
# Set logging for the queries

# Results
unique_docs = multi_retriever.get_relevant_documents(
    query='Who is Allen Turing?'
)
len(unique_docs)

# Other inputs
#question = "What are the approaches to Task Decomposition?"

INFO:langchain.retrievers.multi_query:Generated queries: ['Version 1: What is the Turing Test?', '    Version 2: Who is the father of computer science?', '    Version 3: What is the history of artificial intelligence?', '    Please provide your answer in plain text.', '', '    -----------------------------------------------------------------------------------------', '    Question: Who is Allen Turing?', '    -----------------------------------------------------------------------------------------', '    Version 1:', '    Who is the mathematician who developed the Turing Machine?', '', '    Version 2:', '    Who is the computer scientist who proposed the Turing Test?', '', '    Version 3:', '    Who is the British logician who was convicted of gross indecency for his homosexuality?', '', '    -----------------------------------------------------------------------------------------', '    Please provide your answer in plain text.', '', '    Note: The default context of the topic is lang

19

In [11]:
#compressed_docs = compression_retriever.get_relevant_documents('What is a chain in langchain?')
pretty_print_docs(unique_docs)

Document 1:

angchain/​vectorstores/​prismalangchain/​vectorstores/​qdrantlangchain/​vectorstores/​redislangchain/​vectorstores/​singlestorelangchain/​vectorstores/​supabaselangchain/​vectorstores/​tigrislangchain/​vectorstores/​typeormlangchain/​vectorstores/​typesenselangchain/​vectorstores/​usearchlangchain/​vectorstores/​vectaralangchain/​vectorstores/​voylangchain/​vectorstores/​weaviatelangchain/​vectorstores/​xatalangchain/​vectorstores/​zepAPI referencelangchain/​experimental/​chat_models/​anthropic_functionsClassesAnthropicFunctionsOn this pageAnthropicFunctionsWrapper around Anthropic large language models.To use you should have the@anthropic-ai/sdkpackage installed, with theANTHROPIC_API_KEYenvironment variable set.Remarks​Any parameters that are valid to be passed toanthropic.completecan be passed throughinvocationKwargs,
even if not explicitly available on this class.Hierarchy​ChatAnthropic<ChatAnthropicFunctionsCallOptions>.AnthropicFunctionsConstructors​constructor()​new

In [14]:
from langchain.agents.agent_toolkits import create_conversational_retrieval_agent
from langchain.agents.agent_toolkits import create_retriever_tool
from ibm_watson_machine_learning.foundation_models.extensions.langchain import WatsonxLLM
from ibm_watson_machine_learning.foundation_models.utils.enums import ModelTypes, DecodingMethods
from ibm_watson_machine_learning.metanames import GenTextParamsMetaNames as GenParams
from ibm_watson_machine_learning.foundation_models import Model
from langchain.memory import ConversationBufferMemory, ConversationSummaryMemory

params_temp_0p8 = {
    GenParams.MAX_NEW_TOKENS: 1000,
    GenParams.MIN_NEW_TOKENS: 1,
    GenParams.DECODING_METHOD: DecodingMethods.GREEDY,
    GenParams.TEMPERATURE: 0.8,
    GenParams.TOP_K: 50,
    GenParams.TOP_P: .95
}
credentials = {
    'url': "https://us-south.ml.cloud.ibm.com",
    'apikey' : '5Oz7Ma1ZCctfI0I9FU1kbZC6YRWm9jRKddtL-8mJYREx'
}

project_id = '0ec8f431-3b27-41a3-a540-98d7f62ca1ad'


llama = ModelTypes.LLAMA_2_70B_CHAT

llama_model_temp_0p8 = Model(
model_id=llama,
params=params_temp_0p8,
credentials=credentials,
project_id=project_id)
llm_llama_temp_0p8 = WatsonxLLM(model=llama_model_temp_0p8)

#retriever = db.as_retriever()

from langchain.chains import (
    StuffDocumentsChain, LLMChain, ConversationalRetrievalChain
)
from langchain.prompts import PromptTemplate

# This controls how each document will be formatted. Specifically,
# it will be passed to `format_document` - see that function for more
# details.
document_prompt = PromptTemplate(
    input_variables=["page_content"],
     template="{page_content}"
)
document_variable_name = "context"
#llm = OpenAI()
# The prompt here should take as an input variable the
# `document_variable_name`
sum_prompt = PromptTemplate.from_template(
    "Summarize this content: {context}"
)
sum_llm_chain = LLMChain(llm=llm_llama_temp_0p1, prompt=sum_prompt)
stuff_chain = StuffDocumentsChain(
    llm_chain=sum_llm_chain,
    document_prompt=document_prompt,
    document_variable_name=document_variable_name
)




combine_docs_chain = stuff_chain #StuffDocumentsChain(...)
#retriever =db.as_retriever(search_type="mmr", search_kwargs={'k': 5, 'fetch_k': 50})

# This controls how the standalone question is generated.
# Should take `chat_history` and `question` as input variables.
template = (
    "Combine the chat history and follow up question into "
    "a standalone question. Chat History: {chat_history}"
    "Follow up question: {question}"
)
prompt = PromptTemplate.from_template(template)
#llm = OpenAI()
question_generator_chain = LLMChain(llm=llm_llama_temp_0p8, prompt=prompt)
chain = ConversationalRetrievalChain(
    combine_docs_chain=combine_docs_chain,
    retriever=retriever,
    question_generator=question_generator_chain,
)

In [5]:
from langchain.memory import ChatMessageHistory

history = ChatMessageHistory()
history.add_user_message("hi!")
history.add_ai_message("whats up?")
print(history.messages)

output = chain.run({'chat_history':history.messages,'question':'What is langchain conversational memory?'})
print(output)

[HumanMessage(content='hi!', additional_kwargs={}, example=False), AIMessage(content='whats up?', additional_kwargs={}, example=False)]


NameError: name 'chain' is not defined

In [12]:
import chorma_db

ModuleNotFoundError: No module named 'chorma_db'